# Retrieve Weather Data

In [144]:
# Retrive the weather data
import numpy as np
import pandas as pd
from citipy import citipy
from config import weather_api_key
import requests
import time


#### Creating List of Cities
Created 2,000 random latitudes and longitudes for around 700+ cities

In [145]:
# Create new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = zip(lats,lngs)

In [146]:
# Create list of city names (Using Citipy module)
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

In [147]:
#700+ cities
len(cities)

763

#### Gathering Weather For Each City
Using OpenWeatherMap API

In [148]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [151]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_desc = city_weather['weather'][0]['description']
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_desc})
    
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    
    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")
        

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | skalistyy
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 1 | ravar
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 1 | venice
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 1 | grand river south east
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 1 | nizhneyansk
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 1 | sentyabrskiy
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 1 | 

In [157]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)
    

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ravar,IR,31.2656,56.8055,80.29,18,0,3.56,clear sky
1,Venice,US,27.0998,-82.4543,92.14,73,0,10.36,clear sky
2,Trinidad,BO,-14.8333,-64.9000,70.43,88,100,1.30,overcast clouds
3,Chulumani,BO,-16.4096,-67.5263,63.63,95,99,2.64,overcast clouds
4,Rikitea,PF,-23.1203,-134.9692,69.26,68,92,22.01,overcast clouds
5,Kavieng,PG,-2.5744,150.7967,81.95,74,34,8.25,scattered clouds
6,Hithadhoo,MV,-0.6000,73.0833,81.30,76,59,9.80,broken clouds
7,Busselton,AU,-33.6500,115.3333,51.66,77,67,11.01,broken clouds
8,Mataura,NZ,-46.1927,168.8643,39.47,100,100,2.77,light rain
9,Nikolskoye,RU,59.7035,30.7861,60.80,96,100,5.59,light rain


In [156]:
#Export Dataframe to CSV
city_data_df.to_csv("WeatherPy_Database.csv", index_label="City_ID")